In [17]:
class Tree:
    max_depth=None
    def __init__(self,max_depth):
        self.max_depth=max_depth
    def fit(self,x_train,y_train,feature_names):
        '''
        x_train should be 2-dimension array
        y_train and feature_names should be 1-dimension array
        
        '''
        self.x_train=x_train
        self.y_train=y_train
        self.feature_names=feature_names
        def new_log(n,bottom):
            '''
            caculate log value with bottom,when n=0,log value=0.
        
            '''   
            import math as m
            if n==0:
                return 0
            else:
                return m.log(n,bottom)

        def continuous_encoder(x,border):
            '''
            encode continuous array into 2 catagories:'>border and '<=border'.
        
            '''
            import numpy as np
            n_values=x.shape[0]
            new_x=[]
            for k in range(n_values):
                if x[k]>border:
                    new_x.append(f'>{border:.2f}')
                elif x[k]<=border:
                    new_x.append(f'<={border:.2f}')
            return np.array(new_x)
        
        def entropy(y):
            '''
            Caculate the entropy.  
            y should be an 1-dimension array as target.
            
            '''
            import numpy as np
            catagories,counts=np.unique(y,return_counts=True)
            sample_size=y.shape[0]
            n_catagories=catagories.shape[0]
            entropy=0
            for i in range(n_catagories):
                entropy-=(counts[i]/sample_size)*new_log(counts[i]/sample_size,2)
            return entropy
        
        def info_gain(x,y):
            '''
            caculate the information gain.
            x should be an 1-dimension array as a column of feature.
            y should be an 1-dimension array as target.
            
            '''
            import numpy as np
            x_catagories,x_counts=np.unique(x,return_counts=True)
            sample_size=x.shape[0]
            n_x_catagories=x_catagories.shape[0]
            weights=np.full(n_x_catagories,np.nan)
            catagory_entropies=np.full(n_x_catagories,np.nan)
            for i in range(n_x_catagories):
                weights[i]=x_counts[i]/sample_size
                catagory_entropies[i]=entropy(y[x==x_catagories[i]]) 
            info_gain= entropy(y)-np.sum(weights*catagory_entropies) 
            return info_gain
        
        def continuous_treatment(x,y):
            '''
            discretize an array that is thought to be continuous.
            The output is the discretized array and the border that split the continious array calculated by max entropy.
            
            x,y should be 1-dimension array.
            
            '''
            import numpy as np
            n_type_values=np.unique(x).shape[0]
            n_values=x.shape[0]
            midpoint=np.full(n_values-1,np.nan)
            info_gains=np.full(n_values-1,np.nan)
            sorted_x=np.sort(x)
            for i in range(n_values-1):
                midpoint[i]=(sorted_x[i+1]-sorted_x[i])/2+sorted_x[i]
                info_gains[i]=info_gain(continuous_encoder(x,midpoint[i]),y)
            final_border=np.random.RandomState(123).choice(midpoint[info_gains==np.max(info_gains)])
            return final_border,continuous_encoder(x,final_border)
        
        def best_feature(x_y,feature_names):
            '''
            Return the feature with max info_gain so that the dataframe can be splited next
            x_y should be an 2-dimension array as features and target.
        
            '''
            import numpy as np
            x,y=np.hsplit(x_y,[x_y.shape[1]-1])
            info_gains=np.full(x.shape[1],np.nan)
            for i,featlist in enumerate(x.T):
                if np.unique(x_train[:,i]).shape[0]>=5:
                # if the types of feature in the initial dataframe>=5, the feature is thought to be continous,if continous, discretize it.
                    info_gains[i]=info_gain(continuous_treatment(featlist,y)[1],y)
                else:
                    info_gains[i]=info_gain(featlist,y)
            best_feature_index=np.random.RandomState(123).choice(np.arange(x.shape[1])[info_gains==np.max(info_gains)])
            best_feature=feature_names[best_feature_index]
            return best_feature,best_feature_index

        def data_split(x_y,best_feature_index,catagory):
            '''
            split the dataframe according to the best_feature_indx,and the catagory of best_feature
            
            '''
            import numpy as np
            x,y=np.hsplit(x_y,[x_y.shape[1]-1])
            best_featlist=x[:,best_feature_index]
            if np.unique(x_train[:,best_feature_index]).shape[0]>=5:
                border=continuous_treatment(best_featlist,y)[0]
                if '<=' in catagory:
                    splited_data=x_y[best_featlist<=border]
                elif '>' in catagory:
                    splited_data=x_y[best_featlist>border]
            else:       
                splited_data=x_y[best_featlist==catagory]            
            return splited_data
            
        def new_int(a):
            '''
            used to the final prediction results,to make sure that the results are int if y contains number of str of number
            and the results are text str if y contrains text str.

            '''
            try:
                int(a)
            except:
                return a
            else:
                return int(a) 
                        
        def create_tree(x_y,feature_names,max_depth=-1):
            '''
            create the tree in the form of dictionary

            '''
            import numpy as np
            max_depth+=1
            x,y=np.hsplit(x_y,[x_y.shape[1]-1])
            if np.unique(y).shape[0]==1:
                return str(new_int(np.unique(y)[0]))
            best_feat,best_feat_index=best_feature(x_y,feature_names)
            tree={best_feat:{}}
            best_featlist=x[:,best_feat_index]
            if np.unique(x_train[:,best_feat_index]).shape[0]>=5:
                border=continuous_treatment(best_featlist,y)[0]
                catagories=np.unique(continuous_encoder(best_featlist,border))
            else:
                catagories=np.unique(best_featlist)
            if max_depth==self.max_depth:
                class_list,class_count=np.unique(y,return_counts=True)
                return str(new_int(np.random.RandomState(123).choice(class_list[class_count==np.max(class_count)])))
            elif catagories.shape[0]==1:# When the dataframe can not be splited anymore
                class_list,class_count=np.unique(y,return_counts=True)
                return str(new_int(np.random.RandomState(123).choice(class_list[class_count==np.max(class_count)])))
            else:
                for catagory in catagories:
                    tree[best_feat][catagory]=create_tree(
                    data_split(x_y,best_feat_index,catagory),feature_names,max_depth=max_depth)
                return tree
  
#  Encode data and treat continous data
        import numpy as np
        x_y=np.concatenate((x_train,y_train[:,np.newaxis]),axis=1) 
        tree=create_tree(x_y,feature_names)
        self.tree=tree 
        
    def predict(self,x_test):
        '''
        x_test should be a 2-dimension array

        '''
        def common(x):
            '''
            extract common part between the 2 elements in the 2-size list x

            '''
            result=float(x[0][2:])
            return result
            
        def single_predict(tree,single_x_test):
            '''
            this function predict a test point, x_test should be a dictionary of a single data point
            
            '''
            first=list(tree.keys())[0]
            feature_index=np.arange(len(self.feature_names))[self.feature_names==first]
            copy=single_x_test.copy()
            if np.unique(self.x_train[:,feature_index].shape[0])>=5:
                list_of_lists=list(tree[first].keys())
                border=common(list_of_lists)                
                if single_x_test[first]<=border:
                    copy[first]=f'<={border:.2f}'
                elif single_x_test[first]>border:
                    copy[first]=f'>{border:.2f}' 
            second=copy[first]
            edege=tree[first][second]
            if isinstance(edege,str)==True:
                predict=edege
            else:
                predict=single_predict(edege,single_x_test)
            return predict
        import numpy as np
        empty_x_test=[]
        for i,single_x_test in enumerate(x_test):
            single_x_test=single_x_test.tolist()
            single_x_test={feature_names:single_x_test for feature_names,single_x_test in zip(feature_names,single_x_test)}
            empty_x_test.append(single_predict(self.tree,single_x_test))
        return empty_x_test
        
#test
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import numpy as np
x=load_iris().data
y=load_iris().target
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=123)
feature_names=np.array(load_iris().feature_names)
clf=Tree(3)
clf.fit(x_train,y_train,feature_names)
y_pred=np.array(clf.predict(x_test))
pred_y=np.concatenate((y_pred.astype(int)[:,np.newaxis],y_test[:,np.newaxis]),axis=1)
score=np.unique(y_pred.astype(int)==y_test,return_counts=True)[1][1]/len(y_pred.astype(int)==y_test)
print(f'Decision Tree:\n{clf.tree}\n\n prediction-truth Comparion:\n{pred_y} \n\n score:{score}')

Decision Tree:
{'petal length (cm)': {'<=2.45': '0', '>2.45': {'petal width (cm)': {'<=1.70': {'petal length (cm)': {'<=4.95': '1', '>4.95': '2'}}, '>1.70': {'petal length (cm)': {'<=4.80': '1', '>4.80': '2'}}}}}}

 prediction-truth Comparion:
[[1 1]
 [2 2]
 [2 2]
 [1 1]
 [0 0]
 [1 2]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [2 2]
 [0 0]
 [1 1]
 [2 2]
 [2 2]
 [2 2]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [2 2]
 [0 0]
 [2 2]
 [0 0]
 [0 0]
 [0 0]
 [2 2]
 [2 2]
 [0 0]] 

 score:0.9666666666666667
